In [252]:
from collections import Counter
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 20)

In [254]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [255]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [256]:
from statistic import Statistic
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [257]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [270]:
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [259]:
COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'longitude', 'latitude', 'phone_2_norm']

In [260]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [261]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [262]:
replace_val = -9999
train = train.fillna(replace_val)
test = test.fillna(replace_val)

In [263]:
X_train = train.copy()
y_train = db_teach.status.astype(int)
X_test = test.copy()
y_test = db_test.status.astype(int)

In [264]:
train = train.values
test = test.values
label = db_teach.status

In [265]:
# white = None
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)
weight = analyzer_prediction.get_xgb_weight()

In [266]:
config = {'max_depth': 3, 'learning_rate': 0.2, 'n_estimators': 80 }
model = xgb.XGBClassifier(**config)

In [267]:
model.fit(train, label, eval_metric = 'auc', sample_weight=weight)

XGBClassifier(learning_rate=0.2, n_estimators=80)

In [268]:
# New method for XGBClassifier only with "White list" apply.

In [271]:
db_test['is_white'] = np.where(db_test.id.isin(white.ID), 1, 0)
is_white = db_test.is_white.values

XGBClassifier.is_white = is_white

def predict_proba_with_wl(self, data, ntree_limit=None, validate_features=True):

    test_dmatrix = DMatrix(data, missing=self.missing, nthread=self.n_jobs)
    if ntree_limit is None:
        ntree_limit = getattr(self, "best_ntree_limit", 0)
    class_probs = self.get_booster().predict(test_dmatrix,
                                             ntree_limit=ntree_limit,
                                             validate_features=validate_features)

            
    class_probs =  np.asarray([ 0.0 if self.is_white[i] else class_probs[i] for i in range(len(class_probs))])

    if self.objective == "multi:softprob":
        return class_probs
    else:
        classone_probs = class_probs
        classzero_probs = 1.0 - classone_probs
        return np.vstack((classzero_probs, classone_probs)).transpose()

XGBClassifier.predict_proba_with_wl = predict_proba_with_wl    

In [272]:
from interpret import show
from interpret.perf import ROC
blackbox_perf = ROC(model.predict_proba).explain_perf(X_test, y_test, name='XGBoost')
show(blackbox_perf)

<!-- http://127.0.0.1:7001/140025702594096/ -->

In [273]:
model_perf_wl = ROC(model.predict_proba_with_wl).explain_perf(X_test, y_test, name='XGBoost with apply WL')
show(model_perf_wl)

<!-- http://127.0.0.1:7001/140025487571936/ -->

In [274]:
from interpret.blackbox import LimeTabular
from interpret import show

#Blackbox explainers need a predict function, and optionally a dataset
lime = LimeTabular(predict_fn=blackbox_model.predict_proba, data=X_train, random_state=1)

#Pick the instances to explain, optionally pass in labels if you have them
n = 29276
lime_local = lime.explain_local(X_test[n:n+1], y_test[n:n+1], name='LIME')
show(lime_local)

<!-- http://127.0.0.1:7001/140025485120680/ -->

In [275]:
from interpret.blackbox import ShapKernel
import numpy as np
feature_names = COL_FACTORS
n = 29276
background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict_proba, data=background_val, feature_names=feature_names)
shap_local = shap.explain_local(X_test[n:n+1], y_test[n:n+1], name='SHAP')
show(shap_local)

<!-- http://127.0.0.1:7001/140025485791752/ -->

In [277]:
from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict_proba, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Global Sensitivity")

show(sensitivity_global)

<!-- http://127.0.0.1:7001/140025702593144/ -->

In [278]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict_proba, data=X_train)
pdp_global = pdp.explain_global(name='Partial Dependence')

show(pdp_global)

<!-- http://127.0.0.1:7001/140025365611632/ -->

In [279]:
show([blackbox_perf, lime_local, shap_local, sensitivity_global, pdp_global])

<!-- http://127.0.0.1:7001/140025485939400/ -->
 Open in new window